In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape
from model import CNN_with_mask

In [2]:
def get_padding_data(df):
    '''
    one df represents one game
    split: space, action, player(?), error&score
    label: 發球方是否得分？
    '''
    col = df.columns
    space_col = [c for c in df.columns if 'Space' in c]
    action_col = [c for c in df.columns if 'Action' in c]
    result_col = ['Errors', 'Score', 'Nothing']
    others_col = [c for c in df.columns if c not in space_col and c not in action_col and c not in result_col and c != 'Game' and c != 'Rally']

    team_col = [c for c in df.columns if 'Team' in c]

    rally_set = []
    rally_space_set = []
    rally_action_set = []
    rally_result_set = []
    for _, df_rally in df.groupby(['Game', 'Rally']):   # each rally in one game
        # print(df_rally)
        curr_team = df.iloc[df_rally.index[0]][team_col].tolist()
        shot_set = []
        shot_space_set = []
        shot_action_set = []
        shot_result_set = []
        
        atk_sequence = []
        atk_space_sequence = []
        atk_action_sequence = []
        atk_result = []
        
        for _, shot in df_rally.iterrows():
            if(shot[team_col].tolist() != curr_team):
                shot_set.append(atk_sequence)
                shot_space_set.append(atk_space_sequence)
                shot_action_set.append(atk_action_sequence)
                shot_result_set.append(atk_result)   # 最後一動的結果 -> predict object
                
                curr_team = shot[team_col].tolist()

                atk_sequence = []
                atk_space_sequence = []
                atk_action_sequence = []

            atk_space_sequence.append(shot[space_col])
            atk_action_sequence.append(shot[action_col])
            atk_sequence.append(shot[others_col])
            atk_result = shot[result_col]
        
        # the last shot
        shot_set.append(atk_sequence)
        shot_space_set.append(atk_space_sequence)
        shot_action_set.append(atk_action_sequence)
        shot_result_set.append(atk_result)

        # one rally has been finished
        shot_set = pad_sequences(shot_set, maxlen=3, padding='post')
        shot_space_set = pad_sequences(shot_space_set, maxlen=3, padding='post')
        shot_action_set = pad_sequences(shot_action_set, maxlen=3, padding='post')

        # one rally has been finished
        rally_set.append(shot_set)
        rally_space_set.append(shot_space_set)
        rally_action_set.append(shot_action_set)
        rally_result_set.append(shot_result_set)

    padded_rally_set = pad_sequences(rally_set, dtype=float, padding='post')
    padded_rally_space_set = pad_sequences(rally_space_set, dtype=float, padding='post')
    padded_rally_action_set = pad_sequences(rally_action_set, dtype=float, padding='post')
    padded_rally_result_set = pad_sequences(rally_result_set, dtype=float, padding='post')
    
    return padded_rally_set, padded_rally_space_set, padded_rally_action_set, padded_rally_result_set


In [3]:
df = pd.read_csv('test.csv')

space_replace = {list(df.groupby('Space').groups.keys())[i]: i+1 for i in range(len(df.groupby('Space')))}
action_replace = {list(df.groupby('Action').groups.keys())[i]: i+1 for i in range(len(df.groupby('Action')))}

df = pd.get_dummies(df, columns=['Team', 'No.'])
df = df.replace(space_replace)
df = df.replace(action_replace)

In [4]:
rally_set, rally_space_set, rally_action_set, rally_result_set = get_padding_data(df)

# rally數, 最大回合數in one rally, 3, feature數
print(rally_set.shape)
print(rally_space_set.shape)
print(rally_action_set.shape)
print(rally_result_set.shape)

(1292, 14, 3, 36)
(1292, 14, 3, 1)
(1292, 14, 3, 1)
(1292, 14, 3)


In [5]:
rally_space_set = rally_space_set.squeeze()
rally_action_set = rally_action_set.squeeze()

In [6]:
rally_set_tensor = tf.convert_to_tensor(rally_set)
rally_space_set_tensor = tf.convert_to_tensor(rally_space_set)
rally_action_set_tensor = tf.convert_to_tensor(rally_action_set)
rally_result_set_tensor = tf.convert_to_tensor(rally_result_set)

rally_result_set_tensor = tf.where(tf.math.is_nan(rally_result_set_tensor), 0.0, rally_result_set_tensor)

---

In [7]:
# rally_num = rally_set.shape[0]
rally_size = rally_set.shape[1]
shot_size = 3
feature_dim = (rally_set.shape[-1], len(df.groupby('Space'))+1, len(df.groupby('Action'))+1)
space_embed_size = 8
action_embed_size = 8
shot_embed_size = 16

In [8]:
def create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size):
    '''
    framework: 
    1. 對 space, action 做 embeding, (input, output) = (feature_dim, embed_size)
    2. concat space, action, others 成一個 embedded vector for each atk, (input) =  ([feature_dim, embed_size, embed_size])
    3. 先做 embedding
    4. CNN, filters = shot_embed_size
    5. GRU
    '''
    # each input: 三個維度, rally shot feature
    input_others = keras.Input(shape=(rally_size, shot_size, feature_dim[0]))
    input_space = keras.Input(shape=(rally_size, shot_size))
    input_action = keras.Input(shape=(rally_size, shot_size))

    # space & action 先做 embedding, 再和 others concat
    embed_space_layer = Embedding(input_dim=feature_dim[1], output_dim=space_embed_size, mask_zero=True, name='Space_Embedding')
    embed_action_layer = Embedding(input_dim=feature_dim[2], output_dim=action_embed_size, mask_zero=True, name='Action_Embedding')
    masking_layer = Masking(mask_value=0)   # for input_others (還沒有經過mask)
    concat_layer = Concatenate(name='Input_Concat')

    embed_shot_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=3, name='Shot_Embedding')

    cnn_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=1, name='CNN_Layer')
    gru_layer = GRU(units=16, return_sequences=True, name='GRU_Layer')
    dense_layer = Dense(units=3, activation='softmax')
    reshape_layer = Reshape((-1, 1, 3))

    # forward
    inputs = [input_others, input_space, input_action]

    embed_space = embed_space_layer(input_space)
    embed_action = embed_action_layer(input_action)
    masked_others = masking_layer(tf.cast(input_others, tf.float32))
    embed_input = concat_layer([masked_others, embed_space, embed_action])
    embed_shot = tf.squeeze(embed_shot_layer(embed_input), axis=2)

    cnn_output = cnn_layer(embed_shot)
    gru_output = gru_layer(cnn_output)
    output = dense_layer(gru_output)
    model = keras.Model(inputs=inputs, outputs=output, name='Classification')
    return model

In [9]:
model = create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size)
model.summary()

Model: "Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 14, 3, 36)]  0                                            
__________________________________________________________________________________________________
tf.cast (TFOpLambda)            (None, 14, 3, 36)    0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 14, 3)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 14, 3)]      0                                            
_____________________________________________________________________________________

In [10]:
# print(rally_set_tensor.shape)
# print(rally_space_set_tensor.shape)
# print(rally_action_set_tensor.shape)
# print(rally_result_set_tensor.shape)

In [11]:
# model.call([rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor])

In [12]:
def split_data(others_tensor, space_tensor, action_tensor, label_tensor):
    l = label_tensor.shape[0]
    split_persentage = int(l*0.7)

    train_space = space_tensor[:split_persentage]
    train_action = action_tensor[:split_persentage]
    train_others = others_tensor[:split_persentage]
    train_label = label_tensor[:split_persentage]

    test_space = space_tensor[split_persentage:]
    test_action = action_tensor[split_persentage:]
    test_others = others_tensor[split_persentage:]
    test_label = label_tensor[split_persentage:]

    train_x = [train_others, train_space, train_action]
    train_y = train_label

    test_x = [test_others, test_space, test_action]
    test_y = test_label

    return train_x, train_y, test_x, test_y

In [13]:
train_x, train_y, test_x, test_y = split_data(rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor, rally_result_set_tensor)

In [29]:
# regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
epochs = 30
# callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [30]:
model.fit(train_x, train_y, epochs=epochs)

Epoch 1/30
29/29 [==============================] - 1s 4ms/step - loss: 0.0445 - accuracy: 0.2043
Epoch 2/30
29/29 [==============================] - 0s 5ms/step - loss: 0.0436 - accuracy: 0.2051
Epoch 3/30
29/29 [==============================] - 0s 4ms/step - loss: 0.0439 - accuracy: 0.2038
Epoch 4/30
29/29 [==============================] - 0s 4ms/step - loss: 0.0435 - accuracy: 0.2039
Epoch 5/30
29/29 [==============================] - 0s 4ms/step - loss: 0.0425 - accuracy: 0.2048
Epoch 6/30
29/29 [==============================] - 0s 4ms/step - loss: 0.0425 - accuracy: 0.2047
Epoch 7/30
29/29 [==============================] - 0s 4ms/step - loss: 0.0427 - accuracy: 0.2046
Epoch 8/30
29/29 [==============================] - 0s 4ms/step - loss: 0.0415 - accuracy: 0.2059
Epoch 9/30
29/29 [==============================] - 0s 4ms/step - loss: 0.0414 - accuracy: 0.2057
Epoch 10/30
29/29 [==============================] - 0s 4ms/step - loss: 0.0409 - accuracy: 0.2060
Epoch 11/30
29/29 [

In [31]:
result = model.evaluate(test_x, test_y)

13/13 [==============================] - 0s 1ms/step - loss: 0.0670 - accuracy: 0.1907


In [32]:
y_pred = model.predict(test_x)

In [33]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [57]:
test_y_reshape = test_y.reshape(test_y.shape[0]*test_y.shape[1], 3)
y_pred_reshape = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], 3)

idx = [i for i in range(len(test_y_reshape)) if(any(test_y_reshape[i] == np.array([1, 1, 1])))]
print(len(idx))

1172


In [35]:
argmax_y_pred_reshape = np.argmax(y_pred_reshape[idx], axis=1)
argmax_test_y_reshape = np.argmax(test_y_reshape[idx], axis=1)

In [42]:
from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1, show_eval_result

acc_score = accuracy_score(argmax_test_y_reshape, argmax_y_pred_reshape)
f1_score = f1(argmax_test_y_reshape, argmax_y_pred_reshape)
auc_score = roc_auc_score(test_y_reshape[idx, :], y_pred_reshape[idx, :], multi_class='ovr')
BS = calculate_BS(test_y_reshape[idx, :], y_pred_reshape[idx, :], 3)

In [56]:
show_eval_result(acc_score, f1_score, auc_score, BS)

accuracy:   0.88
f1:         0.97
auc:        0.97
BS:         0.03, 0.07, 0.07
mean of BS: 0.05


---
### Evaluation

In [1]:
from other_model import prosenet_model

In [2]:
model = prosenet_model(shot_sequence_shape=(10, 3, 16))
model.summary()